In [ ]:
import torch

# ==========================================
# 1. 데이터셋 준비 (Next Token Prediction)
# ==========================================
# 언어 모델 학습의 핵심은 "입력된 단어들을 보고, 바로 다음에 올 단어를 맞추는 것"입니다.

# inputs: 모델에게 보여줄 문제 (현재 시점의 단어들)
inputs = torch.tensor([[16833, 3626, 6100],   # 문장 1: ["every", "effort", "moves"]
                       [40,    1107, 588]])   # 문장 2: ["I",     "really", "like"]

# targets: 모델이 맞춰야 할 정답 (한 칸씩 오른쪽으로 이동된 단어들)
# 예: "every"를 보여주면 -> "effort"를 맞춰야 함
targets = torch.tensor([[3626, 6100, 345],    # 문장 1 정답: ["effort", "moves", "you"]
                        [1107, 588, 11311]])  # 문장 2 정답: ["really", "like", "chocolate"]


# ==========================================
# 2. 모델 추론 (Forward Pass)
# ==========================================
# torch.no_grad(): 평가만 할 것이므로 불필요한 기울기(Gradient) 계산 메모리를 아낌
with torch.no_grad():
    # 모델에 입력을 넣어 예측값(Logits)을 얻습니다.
    # logits shape: (배치 크기 2, 문장 길이 3, 단어장 크기 50257)
    # 의미: 2개 문장의 각 3개 위치마다, 50,257개 단어 각각에 대한 점수를 출력
    logits = model(inputs)

In [ ]:
# ==========================================
# 3. 결과 해석 (확률 변환 및 확인)
# ==========================================

# 3-1. 로짓(점수) -> 확률(Probability) 변환
# 로짓은 -무한대 ~ +무한대 범위의 숫자이므로, Softmax를 써서 0~1 사이 확률로 바꿉니다.
# dim=-1: 가장 마지막 차원(단어장 50257개)에 대해 확률의 합이 1이 되게 만듦
probas = torch.softmax(logits, dim=-1) 

print("확률 텐서 크기:", probas.shape) 
# 예상 출력: torch.Size([2, 3, 50257])

# 3-2. 가장 높은 확률을 가진 단어 찾기 (예측 결과)
# argmax: 확률이 가장 높은 인덱스(단어 ID)를 반환
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print("모델이 예측한 토큰 ID:\n", token_ids)

In [ ]:
# ==========================================
# 4. 수동으로 손실(Loss) 계산해보기
# ==========================================
# 목표: 모델이 '정답 단어(Target)'에 대해 얼마나 높은 확률을 부여했는지 확인

# 첫 번째 문장(text_idx=0) 분석
text_idx = 0
# probas[0, 0, targets[0,0]] -> 첫 문장, 첫 단어 위치에서 '정답 단어'의 확률
# [0, 1, 2]: 문장 내의 0번째, 1번째, 2번째 위치를 의미
target_probas_1 = probas[text_idx, [0, 1, 2], targets[text_idx]] # 50257개 중에서 정답인 확률만 뽑음
print("문장 1의 정답 단어들에 대한 예측 확률:", target_probas_1)

# 두 번째 문장(text_idx=1) 분석
text_idx = 1
target_probas_2 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("문장 2의 정답 단어들에 대한 예측 확률:", target_probas_2)

# 4-1. 로그 확률(Log Probability) 계산
# 확률을 그냥 곱하면 숫자가 너무 작아지므로(Underflow), 보통 로그를 취해서 더합니다.
# 모든 정답 확률을 하나로 합칩니다.
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print("정답 확률들의 로그값:", log_probas)

# 4-2. 평균 로그 확률 계산
avg_log_probas = torch.mean(log_probas)
print("평균 로그 확률:", avg_log_probas)

# 4-3. Negative Log Likelihood (NLL)
# 우리는 확률을 '최대화' 하고 싶어하지만, 손실(Loss)은 '최소화' 해야 하는 값입니다.
# 따라서 로그 확률 평균에 -1을 곱해서 양수로 만듭니다. (이게 바로 Cross Entropy Loss의 개념입니다)
neg_avg_log_probas = avg_log_probas * -1
print("수동으로 계산한 손실값(NLL):", neg_avg_log_probas)

### 1. 수식과 코드의 매핑 (Mapping)

이미지의 수식:
$$L = -\sum_{i} y_i \log \hat{y}_i$$

여기서:
* $y_i$ (실제값): 정답인 단어는 1, 나머지는 0 (One-hot Encoding 개념)
* $\hat{y}_i$ (예측값): 모델이 예측한 확률 ($0 \sim 1$)

이 수식이 코드에서 어떻게 변환되는지 보세요.

#### ① $y_i$ (실제값) 처리: `target_probas` 추출
수식에는 $\sum$ 기호가 있어 모든 단어를 다 더하는 것처럼 보이지만, **실제 정답($y_i=1$)이 아닌 단어들은 곱하면 0이 되어 사라집니다.**
즉, **"정답 단어의 확률만 쏙 뽑아내는 것"**이 수식의 핵심입니다.

* **코드:**
    ```python
    # 정답 인덱스(targets)에 해당하는 확률만 인덱싱(Slicing)해서 가져옴
    target_probas_1 = probas[text_idx, [0, 1, 2], targets[text_idx]]
    ```
    이 부분이 수식의 $y_i$ 역할을 수행합니다. 수만 개의 단어 중 정답 단어의 예측 확률($\hat{y}_{\text{target}}$)만 남기는 과정입니다.

#### ② $\log \hat{y}_i$ (로그 취하기): `torch.log`
확률값은 0과 1 사이의 소수입니다. 이를 계속 곱하면 숫자가 너무 작아져서 컴퓨터가 계산을 못하는 '언더플로우(Underflow)'가 발생합니다. 이를 방지하고 계산을 덧셈으로 바꾸기 위해 로그를 취합니다.

* **코드:**
    ```python
    log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
    ```
    이것이 수식의 $\log \hat{y}_i$ 부분입니다.

#### ③ $-$ (마이너스 부호): `* -1`
우리는 정답을 맞출 확률(Log Probability)을 **최대화**하고 싶어 합니다. 하지만 딥러닝 학습(Optimizer)은 손실(Loss)을 **최소화**하는 방향으로 작동합니다.
따라서, "최대화 문제를 최소화 문제로 뒤집기 위해" 마이너스를 붙입니다.

* **코드:**
    ```python
    neg_avg_log_probas = avg_log_probas * -1
    ```
    이것이 수식 맨 앞의 **마이너스($-$)** 기호입니다. 이를 **NLL(Negative Log Likelihood)**라고 부릅니다.

#### ④ $\sum$ (합계/평균): `torch.mean`
수식의 $\sum$은 전체 데이터에 대한 합을 의미합니다. 실제 학습에서는 배치(Batch) 단위로 학습하므로, 합계 대신 **평균(Mean)**을 사용하여 배치 크기에 상관없이 일정한 스케일의 Loss를 유지합니다.

* **코드:**
    ```python
    avg_log_probas = torch.mean(log_probas)
    ```

---

### 2. 요약: 왜 이렇게 계산하나요?

| 수식 요소 | 의미 | 코드 매핑 | 설명 |
| :--- | :--- | :--- | :--- |
| **$y_i$** | 정답만 골라라 | `targets[...]` 인덱싱 | 정답이 아닌 단어는 Loss 계산에서 무시됨 (0이 됨) |
| **$\log \hat{y}_i$** | 확률에 로그 | `torch.log()` | 곱셈을 덧셈으로 변환, 언더플로우 방지 |
| **$\sum$** | 전체 합산 | `torch.mean()` | 배치 내 모든 샘플의 Loss를 종합 |
| **$-$** | 부호 반전 | `* -1` | 확률 최대화(Good)를 Loss 최소화(Good)로 변환 |

In [ ]:
# ==========================================
# 5. PyTorch 함수로 손실(Loss) 계산하기 (권장 방식)
# ==========================================
# 위에서 수동으로 한 과정을 PyTorch는 내부적으로 더 효율적이고 정확하게 처리합니다.

print("-" * 30)
print("로짓 크기 (변경 전):", logits.shape)   # (2, 3, 50257)
print("타깃 크기 (변경 전):", targets.shape)  # (2, 3)

# 5-1. 평탄화 (Flattening)
# CrossEntropyLoss 함수는 입력을 (N, Class) 형태로 받기를 선호합니다.
# 즉, "어떤 문장의 몇 번째 단어인지"는 중요하지 않고, "총 몇 문제를 풀었나"로 형태를 바꿉니다.

# (배치 2 * 길이 3, 단어장 50257) -> (6, 50257) : 총 6개의 단어 예측 문제로 변환
logits_flat = logits.flatten(0, 1)

# (배치 2 * 길이 3) -> (6) : 정답지도 일렬로 6개 나열
targets_flat = targets.flatten()

print("펼친 로짓:", logits_flat.shape)   # torch.Size([6, 50257])
print("펼친 타깃:", targets_flat.shape)  # torch.Size([6])

# 5-2. Cross Entropy Loss 계산
# 주의: nn.functional.cross_entropy는 입력으로 '확률(probas)'이 아니라 '로짓(logits)'을 받습니다.
# 함수 내부적으로 Softmax -> Log -> NLLLoss 과정을 모두 수행하기 때문입니다.
loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)

print("함수로 계산한 손실값(Loss):", loss) 
# 이 값은 위에서 수동으로 계산한 'neg_avg_log_probas'와 거의 같아야 합니다.